# Welcome to the Python CoGe API

The CoGePy API wraps the CoGe (www.genomevolution.org) API with native Python classes and methods to allow easier interaction with the services. This document illustrates the capabilities of the API and should serve as a quick 'getting started' manual.

Coming Soon: Full API scheme.

## Importing Module

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import coge

In [2]:
import pandas as pd
from pprint import pprint as pp

## Authentication

In [5]:
# Create an AuthToken object from a token.
my_auth = coge.auth.AuthToken(username='USERNAME', token='TOKEN')

# Access AuthToken features.
usr = my_auth.username
tkn = my_auth.token

## Organisms

In [ ]:
# Search organisms (by search term).
organism_search_results = coge.organisms.search('Col-0')
#print(organism_search_results)

In [ ]:
# Fetch organism (by ID).
my_organism = coge.Organism(id=1)
my_organism.fetch()
#print(my_organism)

In [ ]:
# Add organism (by Organism())

# Step 1: Create a new Organism() by defining 'name' and 'description'.
# ... TIP: Description MUST be an NCBI Taxonomy (see https://www.ncbi.nlm.nih.gov/taxonomy)
new_organism = coge.Organism(name="Me2Organism", 
                             description='cellular organisms; Eukaryota; Opisthokonta; Metazoa; Eumetazoa; Bilateria;'
                             'Deuterostomia; Chordata; Craniata; Vertebrata; Gnathostomata; Teleostomi; Euteleostomi;'
                             'Sarcopterygii; Dipnotetrapodomorpha; Tetrapoda; Amniota; Mammalia; Theria; Eutheria;'
                             'Boreoeutheria; Euarchontoglires; Primates; Haplorrhini; Simiiformes; Catarrhini;'
                             'Hominoidea; Hominidae; Homininae; Homo; Me2')

# Step 2: 'Add' organism class to CoGe.
# ... TIP: Authentication is always required when adding data to CoGe.
new_org_id = new_organism.add(auth=my_auth)

## Genomes

In [ ]:
# Search genomes (by search term).

# Search public genomes.
public_genome_search_results = coge.genomes.search('Col-0')
#print(public_genome_search_results)

# Search public & private genomes.
private_genome_search_results = coge.genomes.search('Col-0', auth=my_auth)
#print(private_genome_search_results)

In [ ]:
# Fetch genome (by ID).
my_genome = coge.Genome(id=1)
my_genome.fetch(auth=my_auth)
#print(my_genome)

In [ ]:
# Link a genome with an Organism().
my_organism.link_genome(my_genome)

In [ ]:
# Add genome (by Genome()).
# Coming soon!

In [ ]:
# Update genome (by Genome()).
# Coming soon!

In [ ]:
# Fetch sequence (by Genome()).
# Coming soon!

In [7]:
# Export genome (by Genome()).
# ... TIP: dest_type can be set to 'http' or 'irods'. Default is 'irods'.
export_genome = coge.Genome(id=1)
export_genome.export(dest_type='http', auth=my_auth)

https://geco.iplantcollaborative.org/coge-qa/coge/api/v1/genomes/1/export


{u'error': {u'JEX': u'failed to submit workflow'}}

In [ ]:
# Delete genome (by Genome()).
# Coming soon!

## Experiments

In [ ]:
# Search experiments (by search term).

# ... Search public data (without authentication).
anonymous_experiment_search = coge.experiments.search("Col-0")

# ... Search public & your private data (with authentication).
# Coming Soon!
# authorize_organism_search = coge.organisms.search('Col-0', auth=my_auth) 

In [ ]:
# Fetch experiment (by ID).

# ... Public data (without authentication).
public_exp = coge.Experiment(id=1)

# ... Private data (with authentication).
private_exp = coge.Experiment(id=1, auth=my_auth)

In [ ]:
# Add a single experiment (by Experiment()).

# Step 1: Create a local Experiment()
# ... TIP: Required fields are name, genome_id, and source_data. Other fields are either optional, or can be 
#          automatically populated.
my_exp = coge.Experiment(name='My Experiment', 
                         description="An experiment conducted by our excellent team.", 
                         version=1, 
                         source='Lyons Lab', 
                         restricted=True, 
                         genome_id=1, 
                         notebook_id=1, 
                         tags=['Testing', 'Transcriptome'], 
                         additional_metadata=[{'type': 'note', 'text': 'Practice data'},
                                              {'type': 'sequencer', 'text': 'Illumina'}], 
                         source_data=[{'type': 'irods', 'path': '/irods/path/R1.fastq'}, 
                                      {'type': 'irods', 'path': '/irods/path/R2.fastq'}], 
                         analysis_parameters=[{'read_params': {'read_type': 'paired', 'encoding': 33}}, 
                                              {'snp_params': {'method': 'coge'}}])

# Step 2: 'Add' Experiment() to CoGe.
# ... TIP: Authentication is always required when adding data to CoGe.
my_exp.add(auth=my_auth)

In [ ]:
# Bulk add experiments (by [Experiment(), Experiment() ...])

# Load information about experiments from an external spreadsheet.
# ... TIP: See '../tests/experiments.tsv'.
my_data = pd.read_csv('../tests/experiments.tsv', sep='\t', header=0)

# Define analysis parameters.
my_parameters = [{'read_params': {'read_type': 'single', "encoding": 33}},
                 {'alignment_params': {'tool': 'hisat2', 'load_bam': True}}]
                 
# Iterate over experiment data creating Experiments() and appending to list 'my_experiments'
my_experiments = []
for i, d in my_data.iterrows():
    my_exp = coge.Experiment(name=d['name'],
                             description=d['description'],
                             version=d['version'],
                             source=d['source'],
                             restricted=True,
                             genome_id=d['genome_id'],
                             notebook_id=d['notebook_id'],
                             tags=[t for t in d['tags'].split(',')],
                             source_data={'type': 'irods', 'path': d['source_data']},
                             analysis_parameters=my_parameters)
    my_experiments.append(my_exp)

    # Load experiments using the bulk loader.
complete, failed = coge.experiments.bulk_load(my_experiments, auth=my_auth, task_limit=2)

In [ ]:
# Update an experiment (by Experiment()).
old_exp = coge.Experiment(id=1, auth=my_auth)
old_exp.name = "My Renamed Experiment"
update_status = old_exp.update(auth=my_auth)

In [ ]:
# Delete an experiment (by Experiment()).
delete_exp = coge.Experiment(id=1, auth=my_auth)
delete_exp.delete(auth=my_auth)

## Jobs

In [ ]:
# Create a Job() object (from job ID).

# ... Public jobs (no authentication).
public_job = coge.Job(id=1)

# ... Private jobs (authentication required).
private_job = coge.Job(id=1, auth=my_auth)

In [ ]:
# Check/update job status
public_job.update_status()
status = public_job.status